In [ ]:
using JuMP
using GLPK

# Define the number of processes and candidate locations
num_processes = 2
num_locations = 5

# Define the production cost for each location and process
production_cost = [
    1 2 3 4 5;
    6 7 8 9 10
]

# Define the transportation cost between locations for each process
transportation_cost = [
    1 2 3 4 5;
    6 7 8 9 10
]

# Define the demand for each process
demand = [100, 200]

# Define the maximum shipping amount between locations
max_shipping_amount = 50

# Create a JuMP model
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, locations[1:num_processes, 1:num_locations], Bin)
@variable(model, shipping_amount[1:num_processes, 1:num_locations, 1:num_locations] >= 0)

# Define the objective function to minimize the total cost
@objective(model, Min, sum(production_cost[i,j]*locations[i,j] for i in 1:num_processes, j in 1:num_locations) +
                       sum(transportation_cost[i,j]*shipping_amount[i,j,k] for i in 1:num_processes, j in 1:num_locations, k in 1:num_locations))

# Define the constraints
# Each process must have at least two suppliers
for i in 1:num_processes
    @constraint(model, sum(locations[i,j] for j in 1:num_locations) >= 2)
end

# The total amount shipped from each supplier cannot exceed its production capacity
for i in 1:num_processes
    for j in 1:num_locations
        @constraint(model, sum(shipping_amount[i,j,k] for k in 1:num_locations) <= production_capacity[i,j]*locations[i,j])
    end
end

# The total amount received by each process must meet its demand
for i in 1:num_processes
    @constraint(model, sum(shipping_amount[j,k,i] for j in 1:num_processes, k in 1:num_locations) == demand[i])
end

# The shipping amount between suppliers cannot exceed the maximum limit
for i in 1:num_processes
    for j in 1:num_locations
        for k in 1:num_locations
            @constraint(model, shipping_amount[i,j,k] <= max_shipping_amount*locations[i,j])
        end
    end
end

# Solve the model
optimize!(model)

# Print the optimal locations and shipping amounts
println("Optimal locations:")
println(value.(locations))

println("Optimal shipping amounts:")
println(value.(shipping_amount))


In [ ]:
using JuMP, Gurobi

# Define the problem data
num_processes = 2
num_locations = 5
num_products = 1
demand = [100, 100]
production_capacity = [150, 150]
shipping_capacity = [150, 150]
production_cost = [
    [10 12 15 16 18],
    [20 21 23 25 28]
]
transportation_cost = [
    [0 2 3 5 7],
    [2 0 3 4 6]
]

# Create the JuMP model and set the optimizer
model = Model(Gurobi.Optimizer)

# Define the decision variables
@variable(model, x[1:num_processes, 1:num_locations], Bin)
@variable(model, p[1:num_processes, 1:num_locations, 1:num_products] >= 0)
@variable(model, s[1:num_processes, 1:num_locations, 1:num_locations, 1:num_products] >= 0)

# Define the objective function
@objective(model, Min,
    sum(production_cost[i,j]*p[i,j,1] for i in 1:num_processes, j in 1:num_locations) +
    sum(transportation_cost[i,k]*s[i,j,k,1] for i in 1:num_processes, j in 1:num_locations, k in 1:num_locations)
)

# Define the constraints
for i in 1:num_processes
    # Demand constraint
    @constraint(model, sum(p[i,j,1] for j in 1:num_locations) >= demand[i])
    
    for j in 1:num_locations
        # Production capacity constraint
        @constraint(model, p[i,j,1] <= production_capacity[i]*x[i,j])
        
        # Shipping capacity constraint
        @constraint(model, sum(s[i,j,k,1] for k in 1:num_locations) <= shipping_capacity[i]*x[i,j])
        
        for k in 1:num_locations
            # Shipping amount cannot exceed the amount produced
            @constraint(model, s[i,j,k,1] <= p[i,j,1])
        end
    end
    
    # Only one location per process
    @constraint(model, sum(x[i,j] for j in 1:num_locations) == 1)
end

# Solve the model
optimize!(model)

# Print the results
println("Optimal objective value: ", objective_value(model))
println("Selected locations: ")
for i in 1:num_processes
    for j in 1:num_locations
        if value(x[i,j]) > 0.5
            println("Process ", i, " is located at location ", j)
        end
    end
end
println("Production amounts: ")
for i in 1:num_processes
    for j in 1:num_locations
        if value(p[i,j,1]) > 0.1
            println("Process ", i, " produces ", value(p[i,j,1]), " at location ", j)
        end
    end
end
println("Shipping amounts: ")
for i in 1:num_processes
    for j in 1:num_locations
        for k in 1:num_locations
            if value(s[i,j,k,1]) > 0.1
                println("Process ", i, " ships ", value(s[i,j,k,1]), " from location ", j, " to location ", k)
            end
        end
    end
end


In [ ]:
using JuMP, GLPK

# Define the model
m = Model(with_optimizer(GLPK.Optimizer))

# Define the decision variables
# x[i,j,k] = production amount of product k at location i using process j
@variable(m, x[1:5, 1:3, 1:3] >= 0)

# y[i,j,k] = shipment amount of product k from location i to location j using process j
@variable(m, y[1:5, 1:5, 1:3] >= 0)

# Define the objective function
@objective(m, Min, sum(production_cost[i,j,k]*x[i,j,k] for i in 1:5, j in 1:3, k in 1:3) + 
                    sum(shipment_cost[i,j,k]*y[i,j,k] for i in 1:5, j in 1:5, k in 1:3))

# Define the constraints
# Production capacity constraints
@constraint(m, [i in 1:5, k in 1:3], sum(x[i,j,k] for j in 1:3) <= capacity[i,k])

# Product demand constraints
@constraint(m, [i in 1:5, k in 1:3], sum(y[j,i,k] for j in 1:5) + sum(x[i,j,k] for j in 1:3) >= demand[i,k])

# Shipment constraints
@constraint(m, [i in 1:5, j in 1:5, k in 1:3], y[i,j,k] <= max_shipment[i,j,k])

# Process flow constraints
@constraint(m, [i in 1:5, k in 1:3], x[i,2,k] <= x[i,1,k])
@constraint(m, [i in 1:5, k in 1:3], x[i,3,k] <= x[i,2,k])

# Solve the model
optimize!(m)

# Get the optimal solution
println("Optimal solution:")
println("Total Cost: ", JuMP.objective_value(m))
for i in 1:5, j in 1:3, k in 1:3
    println("x[", i, ",", j, ",", k, "] = ", JuMP.value(x[i,j,k]))
end
for i in 1:5, j in 1:5, k in 1:3
    println("y[", i, ",", j, ",", k, "] = ", JuMP.value(y[i,j,k]))
end


In [ ]:
using JuMP, GLPK

# Define the parameters
a = [1 2 3 4 5; 2 3 4 5 6]  # production cost of products 1 and 2 at each location
b = [1 2 3 4; 2 3 4 5]      # shipment cost of product 1 from each location to the next
c = [2 3 4 5; 3 4 5 6]      # shipment cost of product 2 from each location to the next
M = [10 15 20 25 30; 10 15 20 25 30]  # maximum production capacity at each location
N = [5 10 15 20; 5 10 15 20]          # maximum shipment capacity from each location to the next
demand = [50, 50]  # demand for products 1 and 2

# Define the model
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, X[1:5, 1:3, 1:2] >= 0)  # production amount of each product at each location of each process
@variable(model, Y[1:4, 1:2] >= 0)      # shipment amount of product 1 from each location to the next
@variable(model, Z[1:4, 1:2] >= 0)      # shipment amount of product 2 from each location to the next

# Define the objective function
@objective(model, Min, sum(a[i,j,k]*X[i,j,k] for i=1:5, j=1:3, k=1:2) +
                       sum(b[i,j]*Y[i,j] for i=1:4, j=1:2) +
                       sum(c[i,j]*Z[i,j] for i=1:4, j=1:2))

# Define the constraints
# Demand constraint
@constraint(model, X[1,1,1] == demand[1])
@constraint(model, X[3,2,2] == demand[2])
# Production and shipment capacity constraints
for i=1:5, j=1:3, k=1:2
    @constraint(model, X[i,j,k] <= M[i,j])
end
for i=1:4, j=1:2
    @constraint(model, Y[i,j] <= N[i,j])
    @constraint(model, Z[i,j] <= N[i,j])
end
# Flow balance constraints
for j=2:3
    for i=1:5
        if i == 1
            @constraint(model, X[i,j,1] == sum(Y[k,j-1] for k=1:4))
        else
            @constraint(model, X[i,j,1] == X[i-1,j,1] + sum(Y[k,j-1] for k=1:4))
        end
        if i == 5
            @constraint(model, X[i,j,2] == sum(Z[k,j-1] for k=1:4))
        else
            @constraint(model, X[i,j,2] == X[i+1,j,2] + sum(Z[k,j-1] for k=1:4))
        end
    end
end

# Solve the model
optimize!(model)

# Print the results
println("Objective value: ", JuMP.objective_value
